<a href="https://colab.research.google.com/github/itssofiarce/sagrada/blob/main/codigo/extraccion_creditos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!pip install selenium webdriver-manager flask
!pip install beautifulsoup4

'apt-get' is not recognized as an internal or external command,
operable program or batch file.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\marya\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\marya\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium import webdriver
from pathlib import Path


# Configuración más detallada de las opciones de Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')

In [3]:
driver = webdriver.Chrome(options=chrome_options)


In [4]:
def get_credits(delay=5):
    """
    Obtener las stats, con selenium, de la página de BGG de Sagrada
    args:
        delay: int, tiempo de espera para que cargue la página
    returns:
        pd.DataFrame, tabla con las estadísticas.
    """

    driver = webdriver.Chrome(options=chrome_options)

    try:
        url = "https://boardgamegeek.com/boardgame/199561/sagrada/credits"
        driver.get(url)

        print(f"⏳ Esperando {delay} segundos para que carguen los datos...")
        time.sleep(delay)

        stats = driver.find_element(By.CLASS_NAME, "global-body-content-primary.ng-scope")

        # Títulos
        titulos = stats.find_elements(By.CLASS_NAME, "panel-title")
        titulos = [titulo for titulo in titulos if titulo.tag_name == "h3"]
        titulos_texto = [titulo.text for titulo in titulos]

        # Paneles
        paneles = stats.find_elements(By.CLASS_NAME, "panel-body")

        dic_stadisticos = {}

        for titulo, panel in zip(titulos_texto, paneles):
            if titulo not in ["RATINGS BREAKDOWN"]:
                stats_items = panel.find_elements(By.CLASS_NAME, "outline-item")

                for item in stats_items:
                    try:
                        clave = item.find_element(By.CLASS_NAME, "outline-item-title").text.strip()
                        desc = item.find_element(By.CLASS_NAME, "outline-item-description")
                        valores_li = desc.find_elements(By.TAG_NAME, "li")

                        # Si hay <li>, usamos todos los valores
                        if valores_li:
                            valores = [li.text.strip() for li in valores_li]
                        else:
                            valores = [desc.text.strip()]  # Solo un valor

                        dic_stadisticos[clave] = valores

                    except:
                        continue

    finally:
        driver.quit()

    # Convertimos a DataFrame
    df = pd.DataFrame.from_dict(dic_stadisticos, orient='index').reset_index()
    df.columns = ['Credit'] + [f'Value {i+1}' for i in range(df.shape[1] - 1)]
    return df

In [5]:
estadisticas = get_credits()
estadisticas.to_csv("sagrada_stats.csv", index=False)
estadisticas

⏳ Esperando 5 segundos para que carguen los datos...


,Credit,Value 1
0,Primary Name,Sagrada
1,Alternate Names,Саґрада\nСаграда\nซากราดา\nサグラダ\n聖家族大教堂 SAGRAD...
2,Year Released,2017
3,Designers,Adrian Adamescu\nDaryl Andrews
4,Solo Designer,N/A
5,Artist,Peter Wocken
6,Publishers,Floodgate Games\nCranio Creations\nCrowD Games...
7,Developer,N/A
8,Graphic Designer,N/A
9,Sculptor,N/A


## Extracion de relaciones


In [59]:
creditos_csv = pd.read_csv('sagrada_stats.csv')

In [60]:
relaciones = []
def agregar_relacion(sujeto1, relacion, sujeto2):
    relaciones.append({
        "SUJETO1": sujeto1,
        "RELACION": relacion,
        "SUJETO2": sujeto2
    })

In [61]:
# Nombre del juego
juego = creditos_csv.iloc[0, 1]

In [62]:
# Iteramos sobre cada fila para obtener las relaciones
for index, row in creditos_csv.iterrows():
    credit = row['Credit']
    value = row['Value 1']

    # Ignoramos las filas de 'year released' y 'Altername Names'
    if credit in ['Year Released', 'Alternate Names']:
        continue

    # Si el valor no es NaN o vacío, procesamos la relación
    if pd.notna(value) and value != "N/A":

        items = value.split('\n')

        # Usamos el nombre del 'Credit' como la relación
        relacion = credit

        # Agregar una relación por cada elemento en la lista
        for item in items:
            item = item.strip()  # Limpiar espacios extra
            if item:  # Asegurarse de no agregar relaciones vacías
                agregar_relacion(juego, relacion, item)


In [63]:
# Crear el DataFrame con las relaciones
df_relaciones = pd.DataFrame(relaciones)

In [64]:
# Mostrar las primeras relaciones para verificar
print(df_relaciones.head())
print(f"Total de relaciones encontradas: {len(df_relaciones)}")

# Guardar a un CSV
df_relaciones.to_csv("relaciones_sagrada_generadas.csv", index=False)

   SUJETO1      RELACION          SUJETO2
0  Sagrada  Primary Name          Sagrada
1  Sagrada     Designers  Adrian Adamescu
2  Sagrada     Designers    Daryl Andrews
3  Sagrada        Artist     Peter Wocken
4  Sagrada    Publishers  Floodgate Games
Total de relaciones encontradas: 49


## Extracción de relaciones mediante los links

In [65]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

In [66]:
# =================== Ruta al CSV base ===================
ruta_csv = Path('sagrada') / 'codigo' / 'relaciones_sagrada_generadas.csv'
creditos_csv = pd.read_csv(ruta_csv)

In [67]:
# =================== Diccionarios por tipo de categoría ===================
diseñadores_y_artistas = {
    "Daryl Andrews": "https://boardgamegeek.com/boardgamedesigner/67502/daryl-andrews",
    "Adrian Adamescu": "https://boardgamegeek.com/boardgamedesigner/88370/adrian-adamescu",
    "Peter Wocken": "https://boardgamegeek.com/boardgameartist/39902/peter-wocken"
}

editoriales = {
        "Floodgate Games": "https://boardgamegeek.com/boardgamepublisher/21756/floodgate-games",
        "Cranio Creations": "https://boardgamegeek.com/boardgamepublisher/10768/cranio-creations",
        "CrowD Games": "https://boardgamegeek.com/boardgamepublisher/34522/crowd-games",
        "Devir": "https://boardgamegeek.com/boardgamepublisher/2366/devir",
        "FoxGames": "https://boardgamegeek.com/boardgamepublisher/26380/foxgames",
        "Galápagos Jogos": "https://boardgamegeek.com/boardgamepublisher/15605/galapagos-jogos",
        "Geekach Games": "https://boardgamegeek.com/boardgamepublisher/45134/geekach-games",
        "Lanlalen": "https://boardgamegeek.com/boardgamepublisher/36210/lanlalen",
        "Lex Games": "https://boardgamegeek.com/boardgamepublisher/25203/lex-games",
        "Mandoo Games": "https://boardgamegeek.com/boardgamepublisher/30521/mandoo-games",
        "Matagot": "https://boardgamegeek.com/boardgamepublisher/5400/matagot",
        "MEBO Games": "https://boardgamegeek.com/boardgamepublisher/38392/mebo-games",
        "MINDOK": "https://boardgamegeek.com/boardgamepublisher/7992/mindok",
        "MIPL": "https://boardgamegeek.com/boardgamepublisher/51614/mipl",
        "One Moment Games": "https://boardgamegeek.com/boardgamepublisher/30552/one-moment-games",
        "Pegasus Spiele": "https://boardgamegeek.com/boardgamepublisher/39/pegasus-spiele",
        "Reflexshop": "https://boardgamegeek.com/boardgamepublisher/36563/reflexshop",
        "Yes Papa Games": "https://boardgamegeek.com/boardgamepublisher/45523/yes-papa-games"
}

categorias = {
    "Dice": "https://boardgamegeek.com/boardgamecategory/1017/dice",
    "Puzzle": "https://boardgamegeek.com/boardgamecategory/1028/puzzle"
}

mecanicas = {
        "Contracts": "https://boardgamegeek.com/boardgamemechanic/2912/contracts",
        "Dice Rolling": "https://boardgamegeek.com/boardgamemechanic/2072/dice-rolling",
        "End Game Bonuses": "https://boardgamegeek.com/boardgamemechanic/2875/end-game-bonuses",
        "Grid Coverage": "https://boardgamegeek.com/boardgamemechanic/2978/grid-coverage",
        "Open Drafting": "https://boardgamegeek.com/boardgamemechanic/2041/open-drafting",
        "Pattern Building": "https://boardgamegeek.com/boardgamemechanic/2048/pattern-building",
        "Set Collection": "https://boardgamegeek.com/boardgamemechanic/2004/set-collection",
        "Solo / Solitaire Game": "https://boardgamegeek.com/boardgamemechanic/2819/solo-solitaire-game",
        "Turn Order: Progressive": "https://boardgamegeek.com/boardgamemechanic/2828/turn-order-progressive",
        "Variable Set-up": "https://boardgamegeek.com/boardgamemechanic/2897/variable-set-up"
}

familias = {
        "Category: Dized Tutorial": "https://boardgamegeek.com/boardgamefamily/77906/category-dized-tutorial",
        "Cities: Barcelona (Spain)": "https://boardgamegeek.com/boardgamefamily/18639/cities-barcelona-spain",
        "Components: Dice as Playing Units": "https://boardgamegeek.com/boardgamefamily/79497/components-dice-as-playing-units",
        "Crowdfunding: Kickstarter": "https://boardgamegeek.com/boardgamefamily/8374/crowdfunding-kickstarter",
        "Digital Implementations: Google Play": "https://boardgamegeek.com/boardgamefamily/78432/digital-implementations-google-play",
        "Digital Implementations: Steam": "https://boardgamegeek.com/boardgamefamily/77349/digital-implementations-steam",
        "Game: Sagrada": "https://boardgamegeek.com/boardgamefamily/50196/game-sagrada",
        "Mechanism: Dice Drafting": "https://boardgamegeek.com/boardgamefamily/65163/mechanism-dice-drafting",
        "Misc: Watch It Played How To Videos": "https://boardgamegeek.com/boardgamefamily/78198/misc-watch-it-played-how-to-videos",
        "Organizations: The Game Artisans of Canada": "https://boardgamegeek.com/boardgamefamily/4705/organizations-the-game-artisans-of-canada",
        "Players: Games with Solitaire Rules": "https://boardgamegeek.com/boardgamefamily/5666/players-games-with-solitaire-rules",
        "Theme: Art": "https://boardgamegeek.com/boardgamefamily/56419/theme-art",
        "Theme: Art style – Art Nouveau": "https://boardgamegeek.com/boardgamefamily/75454/theme-art-style-art-nouveau",
        "Theme: Stained glass": "https://boardgamegeek.com/boardgamefamily/110914/theme-stained-glass"
}


In [68]:
# =================== Unir todo en un solo diccionario por tipo ===================
perfiles_conocidos = {}

for nombre, url in diseñadores_y_artistas.items():
    perfiles_conocidos[nombre] = ("Diseñador", url)

for nombre, url in editoriales.items():
    perfiles_conocidos[nombre] = ("Editorial", url)

for nombre, url in categorias.items():
    perfiles_conocidos[nombre] = ("Categoría", url)

for nombre, url in mecanicas.items():
    perfiles_conocidos[nombre] = ("Mecánica", url)

for nombre, url in familias.items():
    perfiles_conocidos[nombre] = ("Familia", url)

In [69]:
# =================== Scrap y construcción del DataFrame ===================
relaciones_expandidas = []

for sujeto1, (tipo, url) in perfiles_conocidos.items():
    try:
        driver.get(url)
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        primer_p = soup.find("p")
        texto_completo = primer_p.get_text(strip=True) if primer_p else "Sin párrafo"
        relaciones_expandidas.append({
            "SUJETO1": sujeto1,
            "RELACION": f"Sagrada - {tipo}",
            "SUJETO2": texto_completo
        })
    except Exception as e:
        relaciones_expandidas.append({
            "SUJETO1": sujeto1,
            "RELACION": f"Sagrada - {tipo}",
            "SUJETO2": f"Error accediendo: {e}"
        })

driver.quit()

df = pd.DataFrame(relaciones_expandidas)
df.to_csv("relaciones_mediante_links.csv", index=False)
print(df.head())

            SUJETO1             RELACION  \
0     Daryl Andrews  Sagrada - Diseñador   
1   Adrian Adamescu  Sagrada - Diseñador   
2      Peter Wocken  Sagrada - Diseñador   
3   Floodgate Games  Sagrada - Editorial   
4  Cranio Creations  Sagrada - Editorial   

                                             SUJETO2  
0  Daryl is a game designer and a member of the G...  
1  Adrian is a Game Designer and a member of theG...  
2  I am the Head of Graphic Design at Pandasaurus...  
3                         Microbadge:Floodgate Games  
4  Cranio Creationsis a new and creative Italian ...  
